###Importing Libraries to be used

In [ ]:
import numpy as np
import pandas as pd
import joblib
from pathlib import Path
import os
import shutil
from soundfile import write
import librosa
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
trainframeT = pd.read_csv('/content/gdrive/MyDrive/Freesound_new/train_curated.csv')

Sorted List of all labels

In [ ]:
LABELS=set()
all_labels = list(trainframe['labels'])
for row in all_labels:
    for lab in row.split(r','):
        LABELS.add(lab)
LABELS=list(LABELS)
LABELS.sort()
joblib.dump(LABELS, '/content/gdrive/MyDrive/Freesound/labels.joblib')

['/content/gdrive/MyDrive/Freesound/labels.joblib']

In [ ]:
trainframeT['label_count'] = trainframe['labels'].apply(lambda x: len(x.split(',')))

In [ ]:
trainframeT['label_count'].value_counts()

1    4269
2     627
3      69
4       4
6       1
Name: label_count, dtype: int64

In [ ]:
fname_to_be_deleted = (trainframeT[trainframeT['label_count'] == 6]['fname']).values[0]
fname_path = '/content/gdrive/MyDrive/Freesound/train_curated/' + fname_to_be_deleted
os.remove(fname_path)
fname_to_be_deleted = (trainframeN[trainframeN['label_count'] == 6]['fname']).values[0]
fname_path = '/content/gdrive/MyDrive/Freesound/train_noisy/' + fname_to_be_deleted
os.remove(fname_path)
trainframe = trainframe[trainframe['label_count'] < 6]
y_count=np.array(trainframe['label_count'])
df_train, df_val = train_test_split(df_combined, stratify=y_count, test_size=0.3, random_state=21)

In [ ]:
# Saving for future use
df_train.to_csv(r'/content/gdrive/MyDrive/Freesound_new/df_train.csv', index=False)
df_val.to_csv(r'/content/gdrive/MyDrive/Freesound_new/df_val.csv', index=False)
trainframeT.to_csv('/content/gdrive/MyDrive/Freesound_new/df_combined.csv', index=False)

###Creation of a Custom Configuration Class & Preprocessing Function 

In [ ]:
## taken from kaggle experts
conf = {'sr': 44100, 'duration': 2, 'fmin': 20, 'n_mels': 128}
conf['hop_length'] = 347*conf['duration']
conf['fmax'], conf['n_fft'] = conf['sr'] // 2, conf['n_mels'] * 20
conf['samples'] = conf['sr'] * conf['duration']

In [ ]:
## this code is inspired from https://github.com/divyanshjain19/Freesound_Audio_Tagging
def preprocess(config, data_dir, dest_folder,trim_long_data=False):
    '''
    Description:
    This function performs preprocessing on each audio clip in the given data directory to prepare it for training.
    
    Inputs:
    config: An instance of the Config class that contains configuration parameters for preprocessing.
    data_dir: The path to the directory containing the .wav files to be preprocessed.
    dest_folder: The destination folder where the preprocessed clips will be stored.
    
    Processing:
    1. Load each .wav file in data_dir into a numpy array using a 44 kHz sampling rate.
    2. Trim the leading and trailing silences, i.e., sounds below 60 dB of loudness, from the audio clip.
    3. If the clip is longer than the desired length (specified in config), randomly select a subsample of the clip of the desired length.
       If the clip is shorter than the desired length, pad it with zeroes on both ends to achieve the desired length.
    4. Write the modified clip to the destination folder.
    '''
    for file in os.listdir(data_dir):
        filepath = data_dir + file     
        y, sr = librosa.load(filepath, sr=conf['sr'])
        # trim silence
        if 0 < len(y): # workaround: 0 length causes error
            y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
        # make it unified length to conf.samples
        if len(y) > conf['samples']: # long enough
            if trim_long_data:
                y = y[0:0+conf['samples']]
        else: # pad blank
            padding = conf['samples'] - len(y) # add padding at both ends
            offset = padding // 2
            y = np.pad(y, (offset, conf['samples'] - len(y) - offset), 'constant')
        dest_path = dest_folder + file        
        write(dest_path,y,conf['sr'])

In [ ]:
preprocess(conf, '/content/gdrive/MyDrive/Freesound/train_curated/', '/content/gdrive/MyDrive/Freesound/preprocessed_files_train/')

In [ ]:
preprocess(conf, '/content/gdrive/MyDrive/Freesound/Val/', '/content/gdrive/MyDrive/Freesound/preprocessed_files_val/')

In [ ]:
preprocess(conf, '/content/gdrive/MyDrive/Freesound/test/', '/content/gdrive/MyDrive/Freesound/preprocessed_files_test/')